In [1]:
import os
import time
import json
import numpy as np
import pandas as pd
from scipy import stats
import random
from dotenv import load_dotenv
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [2]:
# Cargar variables de entorno
load_dotenv()
client = MistralClient(api_key="XEV0fCx3MqiG9HqVkGc4Hy5qyD3WwPHr")

In [3]:
# 1. Cargar el corpus de conocimiento
def load_corpus(corpus_path='../data/processed/normalized_data.json'):
    """Carga el corpus de conocimiento desde un archivo JSON"""
    with open(corpus_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [4]:

# 2. Generar preguntas basadas en el corpus
def generate_corpus_questions(corpus, n=30):
    """Genera preguntas relevantes basadas en el corpus usando Mistral AI"""
    # Extraer información clave del corpus para construir un contexto
    cities = set()
    attractions = set()
    unique_titles = set()
    
    for entry in corpus:
        cities.add(entry['city'])
        if entry.get('attractions'):
            attractions.update(entry['attractions'])
        if entry.get('title'):
            # Limpiar título eliminando caracteres especiales
            title = entry['title'].replace('\r', '').replace('\n', '').replace('\t', '').strip()
            if title and len(title) < 100:  # Filtrar títulos muy largos
                unique_titles.add(title)
    
    # Construir contexto para el prompt
    context = f"""
    Estás generando preguntas turísticas sobre Cuba basadas en este contexto:
    - Ciudades mencionadas: {', '.join(cities)}
    - Atracciones turísticas: {', '.join(attractions) if attractions else 'No especificadas'}
    - Lugares destacados: {', '.join(unique_titles) if unique_titles else 'No especificados'}
    
    Genera preguntas que:
    1. Sean relevantes para la información en el corpus
    2. Cubran diferentes aspectos del turismo en Cuba
    3. Sean específicas pero naturales (como las haría un turista)
    4. Incluyan referencias a ciudades, atracciones y conceptos del contexto
    """
    
    # Crear prompt para Mistral AI
    prompt = f"""
    {context}
    
    Genera EXACTAMENTE {n} preguntas sobre turismo en Cuba usando este formato:
    [PREGUNTA 1]
    [PREGUNTA 2]
    ...
    [PREGUNTA {n}]
    
    Reglas:
    - Solo incluye la pregunta sin numeración
    - Usa diferentes tipos de preguntas (qué, dónde, cómo)
    - Refiérete específicamente a: {', '.join(list(cities)[:5])}...
    """
    
    try:
        print("generando preguntas...")
        # Obtener respuesta de Mistral AI
        response = client.chat(
            model="mistral-large-latest",
            messages=[{"role": "user", "content": prompt}]
        )
        
        print("preguntas generadas...")
        
        # Procesar las preguntas generadas
        content = response.choices[0].message.content
        questions = []
        
        for line in content.split('\n'):
            clean_line = line.strip()
            if clean_line and clean_line.endswith('?'):
                questions.append(clean_line)
        
        # Asegurarnos de tener exactamente n preguntas
        if len(questions) >= n:
            return questions[:n]
        else:
            # Generar preguntas de respaldo si no hay suficientes
            base_questions = []
            return base_questions[:n]
            
    except Exception as e:
        print(f"Error generando preguntas: {str(e)}")

# 3. Función para obtener respuestas de tu chatbot
def get_your_chatbot_response(question):
    """
    Obtiene respuesta de TU chatbot (ejecutado manualmente)
    IMPORTANTE: Esta función asume que ejecutarás manualmente tu chatbot
    para las preguntas generadas y guardarás las respuestas en un CSV
    """
    # En la práctica, esto se llenaría manualmente
    return "RESPUESTA DE TU CHATBOT AQUÍ"

In [5]:
# Funciones de procesamiento de respuestas
def extract_content_from_response(response):
    """Extrae el contenido real de una respuesta que puede contener metadatos"""
    if isinstance(response, str):
        if 'content=' in response:
            # Buscar el contenido entre content=' y ', name=
            start = response.find("content='") + 9
            end = response.find("', name=")
            if start > 8 and end > start:
                return response[start:end]
        return response
    return str(response)

def clean_response(resp):
    """Limpia y normaliza una respuesta"""
    if not isinstance(resp, str):
        return ""
    # Limpiar caracteres especiales y espacios extra
    cleaned = resp.strip().replace('\n', ' ').replace('\r', ' ')
    while '  ' in cleaned:
        cleaned = cleaned.replace('  ', ' ')
    return cleaned

def process_response(response):
    """Procesa una respuesta extrayendo el contenido y limpiándolo"""
    return clean_response(extract_content_from_response(response))

In [6]:
# Función de evaluación de similitud actualizada
def evaluate_similarity(response_a, response_b):
    """Evalúa la similitud semántica entre dos respuestas"""
    # Procesar ambas respuestas
    response_a = process_response(response_a)
    response_b = process_response(response_b)
    
    if not response_a or not response_b:
        return 0  # Si alguna respuesta está vacía, la similitud es 0
    
    prompt = f"""
    Evalúa la similitud semántica entre estas dos respuestas sobre turismo en Cuba:
    - Escala: 0 (completamente diferentes) a 10 (idénticas en significado)
    - Considera equivalencia conceptual, no textual
    - Ignora diferencias de formato, estilo o idioma
    - Enfócate en la información turística proporcionada
    
    Respuesta 1: {response_a}
    Respuesta 2: {response_b}
    
    Devuelve SOLO un número entre 0 y 10, sin texto adicional.
    """
    
    try:
        response = client.chat(
            model="mistral-large-latest",
            messages=[ChatMessage(role="user", content=prompt)],
            temperature=0.0,
            max_tokens=10
        )
        return float(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"Error en evaluación de similitud: {str(e)}")
        # Fallback: similitud de coseno básica
        from sklearn.feature_extraction.text import TfidfVectorizer
        vectorizer = TfidfVectorizer().fit_transform([response_a, response_b])
        return ((vectorizer * vectorizer.T).A[0,1] * 10)

# Procesamiento de Respuestas

El notebook ha sido actualizado para manejar correctamente las respuestas del chatbot que incluyen metadatos. Las principales mejoras son:

1. **Extracción de contenido**: Se extrae el contenido real de las respuestas que vienen en formato completo de Mistral
2. **Limpieza de respuestas**: Se normalizan los espacios y caracteres especiales
3. **Evaluación robusta**: La evaluación de similitud ahora maneja mejor diferentes formatos de respuesta
4. **Fallback automático**: Si hay errores en la evaluación principal, se usa una métrica de similitud de coseno como respaldo

Esto asegura que la comparación se realice solo sobre el contenido relevante de las respuestas.

In [15]:
# 4. Chatbot de Mistral AI
def get_mistral_response(question):
    """Obtiene respuesta del modelo Mistral"""
    system_prompt = "Eres un experto en turismo cubano. Proporciona información precisa y útil sobre Cuba."
    
    try:
        response = client.chat(
            model="mistral-large-latest",
            messages=[
                ChatMessage(role="system", content=system_prompt),
                ChatMessage(role="user", content=question)
            ],
            temperature=0.3,
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error en Mistral: {str(e)}")
        return "Error: No se pudo generar respuesta"

In [13]:
# 6. Análisis estadístico con bootstrapping
def analyze_with_bootstrapping(results, n_bootstraps=1000, sample_size=300):
    """Realiza análisis estadístico con bootstrapping"""
    similarity_scores = [res['similarity'] for res in results]
    
    # Estadísticas originales (30 preguntas)
    original_mean = np.mean(similarity_scores)
    original_std = np.std(similarity_scores)
    
    # Bootstrapping para simular muestras más grandes
    bootstrap_means = []
    for _ in range(n_bootstraps):
        sample = np.random.choice(similarity_scores, size=sample_size, replace=True)
        bootstrap_means.append(np.mean(sample))
    
    # Intervalo de confianza
    ci_lower = np.percentile(bootstrap_means, 2.5)
    ci_upper = np.percentile(bootstrap_means, 97.5)
    
    return {
        "original_mean": original_mean,
        "original_std": original_std,
        "bootstrap_mean": np.mean(bootstrap_means),
        "bootstrap_std": np.std(bootstrap_means),
        "ci_95_lower": ci_lower,
        "ci_95_upper": ci_upper,
        "n_bootstraps": n_bootstraps,
        "sample_size": sample_size
    }

In [17]:
# Función principal de evaluación actualizada
def run_evaluation(corpus_path='corpus.json', n_questions=30):
    """Ejecuta el proceso completo de evaluación"""
    # Verificar si ya existen preguntas
    try:
        existing_questions = pd.read_csv('preguntas_evaluacion.csv')
        if not existing_questions.empty:
            questions = existing_questions['pregunta'].tolist()
            print(f"Se encontraron {len(questions)} preguntas existentes en preguntas_evaluacion.csv")
        else:
            raise FileNotFoundError
    except:
        print("Generando nuevas preguntas...")
        corpus = load_corpus(corpus_path)
        questions = generate_corpus_questions(corpus, n_questions)
        pd.DataFrame({'pregunta': questions}).to_csv('preguntas_evaluacion.csv', index=False)
        print(f"Se generaron {len(questions)} preguntas nuevas y se guardaron en preguntas_evaluacion.csv")
        return
    
    # Cargar respuestas manuales
    try:
        manual_df = pd.read_csv('resultados_manuales.csv')
        if len(manual_df) != len(questions):
            raise ValueError(f"El número de respuestas ({len(manual_df)}) no coincide con el número de preguntas ({len(questions)})")
    except Exception as e:
        print(f"Error al cargar resultados_manuales.csv: {str(e)}")
        return
    
    # Procesar preguntas y obtener similitudes
    results = []
    total_questions = len(manual_df)
    print("\nIniciando evaluación comparativa...")
    
    for i, row in manual_df.iterrows():
        print(f"\nProcesando pregunta {i+1}/{total_questions}")
        question = row['pregunta']
        your_response = row['respuesta_manual']
        
        try:
            mistral_response = get_mistral_response(question)
            similarity = evaluate_similarity(your_response, mistral_response)
            
            results.append({
                'pregunta': question,
                'similarity': similarity
            })
            
            print(f"Similitud calculada: {similarity:.2f}/10")
            
        except Exception as e:
            print(f"Error procesando pregunta {i+1}: {str(e)}")
            results.append({
                'pregunta': question,
                'similarity': 0
            })
        
        # Rate limiting
        time.sleep(2.0)
    
    # Análisis estadístico
    analysis = analyze_with_bootstrapping(results)
    
    # Preparar resultados finales
    final_results = {
        'preguntas_totales': total_questions,
        'similitud_promedio': analysis['original_mean'],
        'desviacion_estandar': analysis['original_std'],
        'intervalo_confianza_95_inferior': analysis['ci_95_lower'],
        'intervalo_confianza_95_superior': analysis['ci_95_upper'],
        'resultados_por_pregunta': results
    }
    
    # Guardar resultados
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    output_file = f'resultados_experimento_{timestamp}.json'
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(final_results, f, ensure_ascii=False, indent=2)
    
    print(f"\n✓ Experimento completado")
    print(f"Resultados guardados en: {output_file}")
    print(f"\nEstadísticas:")
    print(f"- Similitud promedio: {final_results['similitud_promedio']:.2f}/10")
    print(f"- Desviación estándar: {final_results['desviacion_estandar']:.2f}")
    print(f"- Intervalo de confianza 95%: [{final_results['intervalo_confianza_95_inferior']:.2f}, {final_results['intervalo_confianza_95_superior']:.2f}]")
    
    return pd.DataFrame(results)

In [ ]:
# 5. Evaluación de similitud semántica
def evaluate_similarity(response_a, response_b):
    """Evalúa la similitud semántica entre dos respuestas"""
    prompt = f"""
    Como experto en evaluación de similitud semántica, analiza estas dos respuestas sobre turismo en Cuba:
    
    Respuesta 1: {response_a}
    Respuesta 2: {response_b}
    
    Evalúa la similitud considerando:
    1. Precisión de la información
    2. Coherencia y relevancia
    3. Cobertura del tema
    4. Calidad de las recomendaciones
    
    Asigna una puntuación:
    - 0-2: Respuestas totalmente diferentes o contradictorias
    - 3-4: Algunas similitudes pero mayormente diferentes
    - 5-6: Similitud moderada, información parcialmente compartida
    - 7-8: Alta similitud en contenido y enfoque
    - 9-10: Prácticamente idénticas en significado y valor informativo
    
    Devuelve SOLO un número entre 0 y 10.
    """
    
    try:
        response = client.chat(
            model="mistral-large-latest",
            messages=[ChatMessage(role="user", content=prompt)]
            max_tokens=10
        )
        return float(response.choices[0].message.content.strip())
    except:
        # Fallback: similitud de coseno básica
        from sklearn.feature_extraction.text import TfidfVectorizer
        vectorizer = TfidfVectorizer().fit_transform([response_a, response_b])
        return ((vectorizer * vectorizer.T).A[0,1] * 10)

In [ ]:
# 7. Flujo principal de evaluación
def run_evaluation(corpus_path='corpus.json', n_questions=30):
    """Ejecuta el proceso completo de evaluación"""
    # Verificar si ya existen preguntas
    try:
        existing_questions = pd.read_csv('preguntas_evaluacion.csv')
        if not existing_questions.empty:
            questions = existing_questions['pregunta'].tolist()
            print(f"Se encontraron {len(questions)} preguntas existentes en preguntas_evaluacion.csv")
        else:
            raise FileNotFoundError
    except:
        print("Generando nuevas preguntas...")
        corpus = load_corpus(corpus_path)
        questions = generate_corpus_questions(corpus, n_questions)
        pd.DataFrame({'pregunta': questions}).to_csv('preguntas_evaluacion.csv', index=False)
        print(f"Se generaron {len(questions)} preguntas nuevas y se guardaron en preguntas_evaluacion.csv")
    
    # Función para extraer el contenido de la respuesta
    def extract_response_content(response_str):
        try:
            if 'content=' in response_str:
                # Buscar el contenido entre content=' y ', name=
                start = response_str.find("content='") + 9
                end = response_str.find("', name=")
                if start > 8 and end > start:  # Si encontramos ambos marcadores
                    return response_str[start:end]
            return response_str
        except:
            return response_str

    # Si ya tenemos respuestas manuales, continuar con la evaluación
    try:
        manual_df = pd.read_csv('resultados_manuales.csv')
        if len(manual_df) != len(questions):
            raise ValueError("Número de respuestas no coincide con preguntas")
            
        # Procesar las respuestas para extraer solo el contenido
        manual_df['respuesta_manual'] = manual_df['respuesta_manual'].apply(extract_response_content)
        
    except Exception as e:
        print(f"Error procesando respuestas manuales: {str(e)}")
        # Crear plantilla para respuestas manuales
        manual_df = pd.DataFrame({
            'pregunta': questions,
            'respuesta_manual': [''] * len(questions)
        })
        manual_df.to_csv('resultados_manuales.csv', index=False)
        return
    
    results = []
    print("\nIniciando evaluación comparativa con Mistral AI...")
    
    for i, row in manual_df.iterrows():
        max_retries = 3
        retry_delay = 2
        
        for attempt in range(max_retries):
            try:
                print(f"Procesando pregunta {i+1}/{len(manual_df)}")
                question = row['pregunta']
                your_response = row['respuesta_manual'].strip()
                
                if not your_response:
                    print(f"Advertencia: Respuesta manual vacía para pregunta {i+1}")
                    continue
                
                mistral_response = get_mistral_response(question)
                if not mistral_response or mistral_response == "Error: No se pudo generar respuesta":
                    print(f"Error: No se pudo obtener respuesta de Mistral para pregunta {i+1}")
                    time.sleep(retry_delay)
                    continue
                
                similarity = evaluate_similarity(your_response, mistral_response)
                
                results.append({
                    'pregunta': question,
                    'respuesta_manual': your_response,
                    'respuesta_mistral': mistral_response,
                    'similarity': similarity
                })
                
                # Rate limiting adaptativo
                time.sleep(2.0)  # Base delay
                break  # Si llegamos aquí, todo salió bien
                
            except Exception as e:
                print(f"Error en pregunta {i+1}, intento {attempt + 1}: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(retry_delay * (attempt + 1))  # Backoff exponencial
                else:
                    print(f"Error máximo de intentos para pregunta {i+1}")
    
    # Análisis estadístico
    analysis = analyze_with_bootstrapping(results)
    
    # Guardar resultados completos
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"resultados_completos_{timestamp}.csv", index=False)
    
    # Guardar análisis estadístico
    analysis_df = pd.DataFrame([analysis])
    analysis_df.to_csv(f"analisis_estadistico_{timestamp}.csv", index=False)
    
    print("\n" + "="*50)
    print("RESULTADOS FINALES")
    print("="*50)
    print(f"Similitud promedio (30 preguntas): {analysis['original_mean']:.2f} ± {analysis['original_std']:.2f}")
    print(f"Similitud estimada ({analysis['sample_size']} preguntas simuladas): {analysis['bootstrap_mean']:.2f}")
    print(f"Intervalo de confianza 95%: [{analysis['ci_95_lower']:.2f}, {analysis['ci_95_upper']:.2f}]")
    
    return results_df, analysis_df


In [12]:
# Ejecutar la evaluación
# NOTA: La primera ejecución genera el CSV para respuestas manuales
# La segunda ejecución (después de completar el CSV) realiza la evaluación completa
results, analysis = run_evaluation(corpus_path='../data/processed/normalized_data.json', n_questions=30)

Se encontraron 30 preguntas existentes en preguntas_evaluacion.csv

Iniciando evaluación comparativa con Mistral AI...
Procesando pregunta 1/30
Procesando pregunta 2/30
Procesando pregunta 2/30
Procesando pregunta 3/30
Procesando pregunta 3/30
Procesando pregunta 4/30
Procesando pregunta 4/30
Procesando pregunta 5/30
Procesando pregunta 5/30
Procesando pregunta 6/30
Procesando pregunta 6/30
Procesando pregunta 7/30
Procesando pregunta 7/30
Procesando pregunta 8/30
Procesando pregunta 8/30
Procesando pregunta 9/30
Procesando pregunta 9/30
Procesando pregunta 10/30
Procesando pregunta 10/30
Procesando pregunta 11/30
Procesando pregunta 11/30
Procesando pregunta 12/30
Procesando pregunta 12/30
Procesando pregunta 13/30
Procesando pregunta 13/30
Procesando pregunta 14/30
Procesando pregunta 14/30
Procesando pregunta 15/30
Procesando pregunta 15/30
Procesando pregunta 16/30
Procesando pregunta 16/30
Procesando pregunta 17/30
Procesando pregunta 17/30
Procesando pregunta 18/30
Procesando pre